#Building a Mortgage Pre-approval AI Agent

## Introduction

This notebook demonstrates how to build a production-ready AI agent that automates the mortgage pre-approval process. We'll follow the six-step framework outlined in our comprehensive guide to building secure AI agents:

1. Define scope and requirements
2. Select models and architecture
3. Create tools and integrations
4. Orchestrate workflows
5. Evaluate performance
6. Deploy to production


To try this on your own, generate a [Cohere API Key here](https://dashboard.cohere.com/welcome/register) and paste this into to notebook in the step 8 below


In [ ]:
!pip install cohere

In [ ]:
# STEP 1: Define scope and setup
# This implementation follows the guide's first step of defining the agent's purpose
# The agent automates the mortgage pre-approval process with these key components:
# - Credit check across three bureaus
# - Employment verification
# - Final report generation

import cohere
import json

# Mock database setup - In production, this would be a real database connection
# This demonstrates the data structure needed for the mortgage pre-approval process
customer_db = {
    "CUS20240111105523": {
        "intake_date": "2024-01-11",
        "customer_id": "CUS20240111105523",
        # Personal information structure - matches guide's data requirements
        "personal_info": {
            "first_name": "Michael",
            "last_name": "Scott",
            "email": "mscott@mail.com",
            "phone": "815-555-5555",
            "address": "123 Drury Lane",
            "city": "Springfield",
            "state": "IL",
            "zip_code": "62701"
        },
        # Document information structure - organized by category
        "document_info": {
            "identity": {
                "ssn": "123-45-6789",
                "dl_number": "M425-7819-3264",
                "dob": "03/15/1965"
            },
            "income": {
                "hourly_rate": 45.67,
                "net_pay": 2643.38,
                "annual_salary": 95000.0,
                "federal_tax_withheld": 19000.0
            },
            "employment": {
                "employer_email": "hr@dundermifflin.com"
            }
        },
        # Empty placeholders for data to be collected during the process
        "credit_info": {},
        "employment_verification": {
            "status": "PENDING"
        }
    }
}

# STEP 2: Tool Creation - Credit Bureau Tools
# These functions implement the guide's tool creation pattern with three components:
# 1. Tool function (the implementation)
# 2. Tool definition (in get_tool_definitions)
# 3. Function mapping (in run_react_agent)

def check_experian(name: str, ssn: str) -> dict:
    """Check credit score with Experian

    Args:
        name (str): Full name of applicant
        ssn (str): Social Security Number

    Returns:
        dict: Credit score information or error message
    """
    try:
        return experian_db[(name, ssn)]
    except KeyError:
        return {"error": "Person not found"}

def check_equifax(name: str, ssn: str) -> dict:
    """Check credit score with Equifax"""
    try:
        return equifax_db[(name, ssn)]
    except KeyError:
        return {"error": "Person not found"}

def check_transunion(name: str, ssn: str) -> dict:
    """Check credit score with TransUnion"""
    try:
        return transunion_db[(name, ssn)]
    except KeyError:
        return {"error": "Person not found"}

# STEP 3: Customer Record Management Tool
def update_customer_record(customer_id: str, credit_info: dict) -> dict:
    """Update customer record with credit information

    Implements the guide's data persistence pattern with error handling
    """
    if customer_id in customer_db:
        customer_db[customer_id]["credit_info"] = credit_info
        return {"status": "success", "message": f"Updated credit info for {customer_id}"}
    return {"error": "Customer not found"}

# STEP 4: Employment Verification Tool
def send_employment_verification_email(employer_name: str, employee_name: str,
                                   employer_email: str, salary: int) -> dict:
    """Send employment verification email

    Implements the guide's external communication pattern with templating
    """
    email_template = f"""Subject: Employment Verification for {employee_name}
    To: {employer_email}
    From: mortgage.processing@bank.com

    Dear {employer_name} HR Team,

    We are processing a mortgage application for {employee_name} and require employment verification.

    Please verify the following information:
    1. Employment Status
    2. Position/Title
    3. Annual Salary (reported: ${salary:,})
    4. Length of Employment

    Your prompt response is appreciated.

    Best regards,
    Mortgage Processing Team"""

    # Debug logging as recommended in the guide
    print("\nSending Employment Verification Email:")
    print("=" * 50)
    print(email_template)
    print("=" * 50)

    # Update verification status
    customer_db["CUS20240111105523"]["employment_verification"].update({
        "status": "PENDING",
        "request_date": "2024-01-11",
        "employer_name": employer_name,
        "reported_salary": salary
    })

    return {"status": "sent", "to": employer_email, "verification_status": "PENDING"}

# STEP 5: Employment Verification Processing
def process_verification_email(email_data: dict) -> dict:
    """Process incoming employment verification email

    Implements the guide's data processing pattern for external responses
    """
    return {
        "status": "VERIFIED",
        "verification_date": email_data["date"],
        "employment_status": "ACTIVE",
        "position": "Regional Manager",
        "salary_verified": 95000,
        "employment_length": "15 years"
    }

def update_employment_verification(customer_id: str, verification_data: dict) -> dict:
    """Update customer record with employment verification results

    Implements the guide's state management with data preservation patterns
    """
    if customer_id in customer_db:
        # Save existing credit info to prevent data loss
        existing_credit_info = customer_db[customer_id].get("credit_info", {})

        # Update employment verification
        customer_db[customer_id]["employment_verification"].update(verification_data)

        # Ensure credit info is preserved
        customer_db[customer_id]["credit_info"] = existing_credit_info

        # Debug logging as recommended in the guide
        print("\nAfter employment verification update:")
        print("Credit Info:", json.dumps(existing_credit_info, indent=2))
        print("Employment Info:", json.dumps(customer_db[customer_id]["employment_verification"], indent=2))

        return {"status": "success", "message": "Employment verification updated"}
    return {"error": "Customer not found"}

# STEP 6: Final Report Generation
def write_final_report(customer_id: str, co_client) -> dict:
    """Generate comprehensive loan officer report

    Implements the guide's report generation pattern using LLM
    """
    customer = customer_db[customer_id]

    # Create detailed context for the LLM
    prompt = f"""You are an expert mortgage analyst responsible for creating detailed loan application review reports for loan officers. Your reports help loan officers make informed decisions about mortgage applications.

    Current application details:
    Applicant: {customer['personal_info']['first_name']} {customer['personal_info']['last_name']}
    ID: {customer_id}
    Credit Scores: Experian ({customer['credit_info'].get('experian', 0)}), Equifax ({customer['credit_info'].get('equifax', 0)}), TransUnion ({customer['credit_info'].get('transunion', 0)})
    Employment Status: {customer['employment_verification'].get('status', 'PENDING')}
    Employment Details: {json.dumps(customer['employment_verification'], indent=2)}
    Income Information: {json.dumps(customer['document_info']['income'], indent=2)}
    Contact Info: {json.dumps(customer['personal_info'], indent=2)}

    Write a comprehensive mortgage application review report for a loan officer that:
    1. Analyzes the credit profile and its implications
    2. Evaluates the employment verification status
    3. Reviews income and tax information
    4. Highlights potential concerns or positive factors
    5. Provides context for decision making"""

    messages = [{"role": "user", "content": prompt}]

    # Generate report using Cohere's Command model
    response = co_client.chat(
        model="command-r-plus-08-2024",
        messages=messages
    )

    return {
        "loan_officer_report": response.message.content[0].text
    }

# STEP 7: Tool Definitions
def get_tool_definitions(customer_id):
    """Define available tools for the agent

    Implements the guide's tool definition pattern with clear descriptions
    and parameter specifications
    """
    return [
        {
            "type": "function",
            "function": {
                "name": "check_experian",
                "description": "Check Experian credit score",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "name": {"type": "string"},
                        "ssn": {"type": "string"}
                    },
                    "required": ["name", "ssn"]
                }
            }
        },
        # Additional tool definitions...
    ]

# STEP 8: Main Agent Implementation
def run_react_agent(customer_id, max_steps=10, verification_email=None):
    """Main agent orchestration function

    Implements the guide's ReAct pattern for sequential workflow handling
    """
    # Initialize Cohere client
    co_client = cohere.ClientV2(api_key="your-api-key")
    customer = customer_db[customer_id]

    # Define system message with clear instructions
    system_message = f"""You are a mortgage pre-approval agent with access to customer data:
    Customer ID: {customer_id}
    Name: {customer['personal_info']['first_name']} {customer['personal_info']['last_name']}
    SSN: {customer['document_info']['identity']['ssn']}
    Salary: {customer['document_info']['income']['annual_salary']}

    Follow these steps:
    1. Check credit scores from all bureaus
    2. Update customer record with credit info
    3. Send employment verification email
    4. Once employment is verified, write final report"""

    # Initialize conversation
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": f"Process pre-approval for {customer['personal_info']['first_name']} {customer['personal_info']['last_name']}"}
    ]

    # Handle employment verification if provided
    if verification_email:
        print("\n" + "=" * 50)
        print("Processing employer verification response...")
        print("=" * 50 + "\n")

        verification_result = process_verification_email(verification_email)
        update_result = update_employment_verification(customer_id, verification_result)

        print("\n" + "=" * 50)
        print("Employer verification complete. Generating final report...")
        print("=" * 50 + "\n")

        messages.append({
            "role": "user",
            "content": "Employment verification received and processed. Please write the final report."
        })

    # Setup tools and function mapping
    tools = get_tool_definitions(customer_id)
    functions_map = {
        "check_experian": check_experian,
        "check_equifax": check_equifax,
        "check_transunion": check_transunion,
        "update_customer_record": update_customer_record,
        "send_employment_verification_email": send_employment_verification_email,
        "write_final_report": write_final_report
    }

    # Main agent loop
    step = 0
    last_step = None
    while step < max_steps:
        # Get next action from LLM
        response = co_client.chat(
            model="command-r-plus-08-2024",
            messages=messages,
            tools=tools
        )

        # Handle tool calls
        if response.message.tool_calls:
            messages.append({
                "role": "assistant",
                "tool_calls": response.message.tool_calls,
                "tool_plan": response.message.tool_plan
            })

            # Print execution plan at first step
            if step == 0:
                print("\n" + "=" * 50)
                print("EXECUTION PLAN:")
                print("=" * 50)
                print(response.message.tool_plan)
                print("=" * 50 + "\n")

            # Execute tools
            for tool_call in response.message.tool_calls:
                if tool_call.function.name != last_step:
                    print("\n" + "-" * 30)

                params = json.loads(tool_call.function.arguments)

                # Special handling for final report
                if tool_call.function.name == "write_final_report":
                    result = write_final_report(params["customer_id"], co_client)
                else:
                    result = functions_map[tool_call.function.name](**params)
                    print(f"Executing: {tool_call.function.name}")
                    print(f"Parameters: {json.dumps(params, indent=2)}")
                    print(f"Result: {json.dumps(result, indent=2)}")

                last_step = tool_call.function.name

                # Update conversation history
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": [{"type": "document", "document": {"data": json.dumps(result)}}]
                })
        else:
            # Handle completion or waiting state
            if not verification_email:
                print("\n" + "=" * 50)
                print("Waiting for employer verification response...")
                print("=" * 50)
            return response.message.content[0].text

        step += 1

    return "Max steps reached"

# Main execution
if __name__ == "__main__":
    # Initial processing
    result = run_react_agent("CUS20240111105523")

    # Test with employment verification
    incoming_email = {
        "from": "hr@dundermifflin.com",
        "subject": "RE: Employment Verification Request for Michael Scott",
        "date": "2024-01-11",
        "body": """Dear Mortgage Processing Team,

        In response to your employment verification request for Michael Scott:
        1. Employment Status: ACTIVE (Full-Time)
        2. Position/Title: Regional Manager
        3. Annual Salary: 95000
        4. Length of Employment: 15 years

        Best regards,
        Toby Flenderson
        HR Department
        Dunder Mifflin Paper Company"""
    }

    result = run_react_agent("CUS20240111105523", verification_email=incoming_email)
    print(result)


EXECUTION PLAN:
I will first check Michael Scott's credit scores with all three bureaus. I will then update the customer record with the credit info. I will then send an employment verification email to hr@dundermifflin.com. Once employment is verified, I will write a final report for the loan officer.


------------------------------
Executing: check_experian
Parameters: {
  "name": "Michael Scott",
  "ssn": "123-45-6789"
}
Result: {
  "credit_score": 720,
  "report_date": "2024-01-10"
}

------------------------------
Executing: check_equifax
Parameters: {
  "name": "Michael Scott",
  "ssn": "123-45-6789"
}
Result: {
  "credit_score": 715,
  "report_date": "2024-01-09"
}

------------------------------
Executing: check_transunion
Parameters: {
  "name": "Michael Scott",
  "ssn": "123-45-6789"
}
Result: {
  "credit_score": 718,
  "report_date": "2024-01-11"
}

------------------------------
Executing: update_customer_record
Parameters: {
  "customer_id": "CUS20240111105523",
  "credi